In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf
import pathlib
import requests
import urllib
import time
import hashlib
import tensorflow_hub as hub

from bs4 import BeautifulSoup
from tensorflow import keras

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from keras.optimizers import *
from keras.losses import sparse_categorical_crossentropy

In [ ]:
# url = "https://www.istockphoto.com/search/2/image?mediatype=photography&page=3&phrase=cruise%20ship&sort=best"
# headers = {
#     'Accept-Encoding': 'gzip, deflate, sdch',
#     'Accept-Language': 'en-US,en;q=0.8',
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36',
#     'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
#     'Referer': 'http://www.wikipedia.org/',
#     'Connection': 'keep-alive',
# }

# get_url = requests.get(url=url,headers=headers).text
# # print(get_url)

# soup = BeautifulSoup(get_url,'lxml')

# images = []

# img_links = soup.select('img[src^="https://media.istockphoto.com/photos"]')
# # print(img_links)

# for i in range(len(img_links)):
#     images.append(img_links[i]['src'])

# for i in range(len(images)):
#     ships = "D:/mixed images/ships/"+str(i)+".jpg"
#     urllib.request.urlretrieve(images[i],ships)

In [ ]:
data_dir = pathlib.Path('D:/mixed images')
data_dir

In [ ]:
# hashlist=[]
# unique = dict()
# for img in data_dir.glob('*/*.jpg'):
#     filehash = hashlib.md5(open(img,"rb").read()).hexdigest()
#     hashlist.append(filehash)

    
#     if filehash in unique:
#         os.remove(img)
#     else:
#         unique[filehash] = data_dir

In [ ]:
len(list(data_dir.glob('*/*.jpg')))

In [ ]:
# planes = list(data_dir.glob('airplanes/*.jpg'))

In [ ]:
# PIL.Image.open(planes[10])

In [ ]:
img_list = [list(data_dir.glob('airplanes/*')),
            list(data_dir.glob('animals/*')),
            list(data_dir.glob('birds/*')),
            list(data_dir.glob('cars/*')),
            list(data_dir.glob('flowers/*')),
            list(data_dir.glob('people/*')),
            list(data_dir.glob('ships/*')),
            list(data_dir.glob('traffic_signs/*')),
            list(data_dir.glob('trains/*'))]

In [ ]:
obj_list = os.listdir(data_dir)

obj_img_dict = dict(zip(obj_list,img_list))

obj_label_dict = dict(zip(obj_list,[0,1,2,3,4,5,6,7,8]))
obj_label_dict

In [ ]:
X = []
y = []

for image_name,images in obj_img_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(299,299))
        X.append(resized_img)
        y.append(obj_label_dict[image_name])

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.35,random_state=42)

In [ ]:
X_train_scaled = X_train/255
X_test_scaled = X_test/255

In [ ]:
obj_classes = 9
model = keras.Sequential()

# building the convolution layers
model.add(keras.layers.Conv2D(32,(3,3),input_shape= (299,299,3),padding='same',activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(32,(3,3), padding='same',activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(64,(3,3), padding='same',activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(64,(3,3), padding='same',activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.BatchNormalization())
# model.add(keras.layers.Conv2D(128,(3,3), padding='same',activation='relu'))
# model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
# model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Flatten())
    
# building the dense layers
model.add(keras.layers.Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(keras.layers.Dense(128, activation='relu'))
# model.add(Dropout(0.5)) # optional
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(obj_classes,activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.fit(X_train_scaled, y_train, batch_size=64,epochs=40, verbose=2)

In [ ]:
model.evaluate(X_test_scaled,y_test)

In [ ]:
import tarfile
tar_file = tarfile.open("D:\imagenet_inception_v3_feature_vector_5.tar.gz")
tar_file.extractall('D:\inception_v3')
tar_file.close()

In [ ]:
get_features = keras.models.load_model('D:\inception_v3')

In [ ]:
inception_model_vectors = hub.KerasLayer(get_features,input_shape=(299,299,3),trainable=False)

In [ ]:
obj_classes = 9
inceptionv3_model = keras.Sequential()
inceptionv3_model.add(inception_model_vectors)
inceptionv3_model.add(keras.layers.Dense(obj_classes,activation='softmax'))

inceptionv3_model.summary()

In [ ]:
inceptionv3_model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
inceptionv3_model.fit(X_train_scaled, y_train, batch_size=32,epochs=10, verbose=2)

In [ ]:
inceptionv3_model.evaluate(X_test_scaled,y_test)